In [2]:
import sys
from concurrent import futures
import numpy as np
from pyspark.sql import SparkSession
from datetime import datetime,timedelta
import pytz

# print(datetime.now(pytz.timezone("Asia/Shanghai")).strftime( '%Y-%m-%d'))
today=(datetime.now()+ timedelta(-1)).strftime( '%Y-%m-%d')
yesterday=(datetime.now(pytz.timezone("Asia/Shanghai"))+ timedelta(-1)).strftime( '%Y-%m-%d')
# ,"banda_stream_etl":"1"
tablemap={
    "banda":{
       
         "t_authorized_channel1_detail":"""      with authorized_detail as      (      SELECT  auto_loan.loan_id    ,case when w.customer_id is not null  then 'repayday'         when w.customer_id is null and detail.real_policy='payday_amount_scorecard' then 'payday'         else 'repayday' end as  acct_type        ,detail.finish_time + interval '7' hour       AS authorized_time    ,unix_timestamp(detail.finish_time)- unix_timestamp(cc.create_time)  as duration       ,detail.customer_id    ,cast(get_json_object(detail.raw_data,'$.maxAmount') AS int)/14000 AS max_authorized_amt       FROM       (       SELECT  loan_id     ,min(id) AS id       FROM `banda-etl-s3`.t_auto_review_loan   WHERE real_policy in('payday_amount_scorecard', 'repayday_amount_scorecard')   AND result = 'PASS'   AND create_time > date '2019-08-01'   GROUP BY  loan_id    )auto_loan      left jOIN  `banda-etl-s3`.t_auto_review_loan detail      ON auto_loan.id = detail.id      LEFT JOIN `banda-etl-s3`.t_loan_app cc      ON auto_loan.loan_id=cc.id      left join ( SELECT customer_id FROM `banda_analysis_etl`.t_customer_detail where is_whitelist = 'Y' ) w on detail.customer_id=w.customer_id      where detail.customer_id not IN ( select customer_id from `banda-etl-s3`.t_test_customer)      ),     withdraw_detail as     (    SELECT  loan_app_id       ,create_time + interval '7' hour AS withdraw_time    FROM `banda-etl-s3`.t_loan_app_status_log    WHERE old_status = 'AUTHORIZATION'     AND new_status = 'FINAL_REVIEW'     AND create_time > date'2019-08-01'      ),    channel1_load_detail AS     (    SELECT  *    FROM `id_adapundi_data_input`.`adapundi_channel_input_data`    WHERE create_time IN ( SELECT MAX(create_time) FROM `id_adapundi_data_input`.`adapundi_channel_input_data` where  substr(create_time,1,1)='2'   )     )     SELECT  aa.*       ,CASE WHEN cc.channel1 is not null THEN cc.channel1  ELSE 'Others' END AS channel      ,dd.withdraw_time      ,now() as insert_time    FROM authorized_detail aa    LEFT JOIN     (   SELECT  customer_id      ,coalesce(media_source,'None') AS media_source      ,coalesce(agency,'None') AS agency   FROM `banda-etl-s3`.t_customer_install_info     ) bb    ON aa.customer_id=bb.customer_id    LEFT JOIN channel1_load_detail cc    ON bb.media_source = cc.mediasource AND bb.agency = cc.agency     LEFT JOIN withdraw_detail dd    ON aa.loan_id = dd.loan_app_id """  ,
        "t_distribute_detail_daily" :  """

                -- 记录每个案件每次分案的信息
                -- 一个案件在一个队列内部流转，对于队列来说计算最早那次流入算作流入队列时间，但是对于催收员来说，仅计算最后一次队列内流转的分案信息
                -- 需要剔除分案之前结清的订单
                -- 月初流转的数据怎么计算：月初流转以后进行判断，如果可以流转到下个队列就流转（fst_assign_ingroup_time），如果不可以，就停留在当前队列（final_assign_ingroup_time）
                with paid_off_detail as 
                (
                SELECT  lg.loan_app_id
                       ,lg.create_time AS paid_off_time
                FROM `banda-etl-s3`.t_loan_app app2
                JOIN `banda-etl-s3`.t_loan_app_status_log lg
                ON lg.loan_app_id=app2.id AND lg.new_status='PAID_OFF'
                WHERE app2.product_name='RUPIAHONE'
                AND app2.status IN ('CURRENT', 'GRACE_PERIOD', 'OVERDUE', 'PAID_OFF')  
                ),
                assign_detail as 
                (
                    select*from
                    (
                    SELECT  new_assign.*
                           ,row_number()OVER (PARTITION BY ref_id,date(create_time+interval'7'hour) ORDER BY create_time DESC )AS rn
                           ,paid_off.paid_off_time --结清时间（分案之后的）
                    FROM `banda-etl-s3`.t_case_distribution_log new_assign
                    left join paid_off_detail paid_off on  new_assign.ref_id=paid_off.loan_app_id
                    WHERE topic='AVERAGE_ASSIGN_CASE_TO_REVIEWER'
                    and  (create_time<paid_off_time or  paid_off.loan_app_id is null)--剔除分案之前已结清分案
                    ) 
                    where rn=1 --月初可能出现一天内分案两次的情况，取最后一次为准
                ),
                audit_final as
                (
                    select*from 
                    (
                    SELECT  ref_id                                           AS loan_app_id
                           ,reviewer_id                                      AS collector_id
                           ,trigger
                           ,create_time as final_assign_ingroup_time -- 组内最后一次分案时间(当月)
                           ,month(create_time+interval'7'hour) as assign_month
                           ,paid_off_time
                           ,row_number()OVER (PARTITION BY ref_id,trigger,month(create_time+interval'7'hour) ORDER BY create_time DESC )AS rn
                    FROM assign_detail
                    )
                    where rn=1
                ),
                audit_fst as 
                (	select 
                    ref_id
                    ,trigger
                    ,month(create_time+interval'7'hour) as assign_month
                    ,min(create_time) as fst_assign_ingroup_time  -- 首次进入组内分案时间
                    FROM assign_detail
                    group by 
                    ref_id
                    ,trigger
                    ,month(create_time+interval'7'hour)
                ), 
                audit_detail as 
                (
                select
                a.*,b.fst_assign_ingroup_time
                from audit_final a 
                left join audit_fst b on a.loan_app_id=b.ref_id and a.trigger=b.trigger and a.assign_month=b.assign_month
                ),
                assign_detail_nosold as -- 剔除分案并且被出售记录
                (
                    SELECT  t1.loan_app_id
                           ,t1.collector_id as final_assign_ingroup_collector_id
                           ,t1.full_name_x  as final_assign_ingroup_collector
                           ,trigger
                           ,t1.amount
                           ,t1.fst_assign_ingroup_time
                           ,t1.final_assign_ingroup_time
                           ,t1.overdue_assign
                           ,t1.id
                           ,t1.erase_amount
                           ,t1.paid_off_time
                    FROM
                    (
                        SELECT  audit.loan_app_id
                               ,audit.collector_id                                                                                       
                               ,ad.full_name_x                                                                                           
                               ,audit.fst_assign_ingroup_time
                               ,audit.final_assign_ingroup_time
                               ,audit.paid_off_time
                               ,app.amount
                               ,lpay.due_date
                               ,lpay.id
                               ,lpay.erase_amount
                               ,trigger
                               ,datediff(DATE(audit.fst_assign_ingroup_time+interval'7'hour),DATE(lpay.due_date+interval'7'hour)) AS overdue_assign --分案时逾期天数
                        FROM audit_detail audit 
                        LEFT JOIN`banda-etl-s3`.t_admin ad
                        ON ad.id=audit.collector_id
                        LEFT JOIN `banda-etl-s3`.t_loan_app app
                        ON app.id=audit.loan_app_id
                        LEFT JOIN `banda-etl-s3`.t_lpay lpay
                        ON lpay.loan_app_id=audit.loan_app_id
                        LEFT JOIN
                        (
                            SELECT  rel.loan_id
                            FROM `banda-etl-s3`.t_loan_tag_rel rel
                            WHERE rel.tag_id IN ( SELECT id FROM `banda-etl-s3`.t_tag WHERE type='SELLING_CASE' )
                            AND rel.status='ACTIVE' 
                        ) mba
                        ON mba.loan_id=audit.loan_app_id
                        LEFT JOIN `banda-etl-s3`.t_collection_blacklist blacklist
                        ON blacklist.customer_id=app.customer_id AND blacklist.status='ACTIVE'
                        WHERE blacklist.customer_id IS NULL
                        AND mba.loan_id IS NULL -- 出售资产 
                    )t1
                )

                -- 建立分案信息明细表 日全量刷新  t_distribute_detail_daily
                SELECT  t3.*
                       ,CASE WHEN t3.paid_prin_before_assign IS NULL THEN t3.loan_amt  ELSE t3.loan_amt-t3.paid_prin_before_assign END AS assign_unpaid_amount -- 剔除在分案之前已还款的金额
                       ,now() as insert_time
                FROM
                (
                    SELECT  t2.loan_app_id
                           ,t2.final_assign_ingroup_collector_id
                           ,t2.final_assign_ingroup_collector
                           ,t2.overdue_assign                            AS overduedays_assign
                           ,t2.amount                                    AS loan_amt
                           ,t2.fst_assign_ingroup_time+interval'7'hour   AS fst_assign_ingroup_time_ind
                           ,t2.final_assign_ingroup_time+interval'7'hour AS final_assign_ingroup_time_ind
                           ,t2.paid_off_time+interval'7'hour             AS paid_off_time_ind
                           ,if(map_load.enum is not null,map_load.map,trigger_group.description) as assign_stage
                           ,t2.trigger
                           ,SUM(deposit.cleared_amount)                  AS paid_amt_before_assign
                           ,SUM(clear.principal)                         AS paid_prin_before_assign
                    FROM assign_detail_nosold t2
                    LEFT JOIN `banda-etl-s3`.t_lpay_deposit deposit
                    ON deposit.lpay_id=t2.id AND deposit.status='CLEARED' AND deposit.create_time<t2.fst_assign_ingroup_time 
                    AND ( deposit.deposit_method !='DIRECT_TRANSFER' OR (deposit.deposit_method ='DIRECT_TRANSFER' AND deposit.cleared_amount !=ceil(t2.erase_amount)))
                    LEFT JOIN `banda-etl-s3`.t_reduce reduce
                    ON reduce.deposit_id =deposit.id AND reduce.reduce_status='SUCCEED'
                    LEFT JOIN `banda-etl-s3`.t_clear_detail_log clear
                    ON clear.deposit_id=deposit.id
                    LEFT JOIN (select*from banda_rpt_mid.enum_mapping_load_table where id='1001') map_load  on t2.trigger=map_load.enum
                    LEFT JOIN
                    (select*from (
                        SELECT  trigger
                               ,description
                               ,row_number()over(partition by trigger ORDER BY update_time desc)rn
                        FROM `banda-etl-s3`.t_trigger_group )
                        WHERE rn=1 
                    ) trigger_group
                    ON t2.trigger=trigger_group.trigger
                    WHERE (reduce.reduce_type IS NULL OR reduce.reduce_type NOT IN ('DELAY_CALLBACK', 'NONE_CALLBACK_REDUCE', 'COLLECTION_REDUCE'))
                    GROUP BY  t2.loan_app_id
                             ,t2.final_assign_ingroup_collector_id
                             ,t2.final_assign_ingroup_collector
                             ,t2.overdue_assign
                             ,t2.amount
                             ,t2.fst_assign_ingroup_time+interval'7'hour
                             ,t2.final_assign_ingroup_time+interval'7'hour
                             ,t2.paid_off_time+interval'7'hour
                             ,if(map_load.enum is not null,map_load.map,trigger_group.description)
                             ,t2.trigger
                )t3 """,
        "t_assign_detail_stage_daily":"""           
            -- 建立每日全量刷新表:分案明细对应阶段表
            -- banda_rpt_mid.t_assign_detail_stage_daily 
            SELECT  assign.*
                   ,admin.full_name_x
                   ,if(map_load.enum is not null,map_load.map,trigger_group.description) AS assign_stage
                   ,now() as insert_time
            FROM
            (
                SELECT  new.ref_id                                                                 AS loan_app_id
                       ,new.admin_id
                       ,new.reviewer_id                                                            AS collector_id
                       ,new.create_time
                       ,new.topic
                       ,trigger
                       ,row_number()OVER (PARTITION BY ref_id ORDER BY create_time DESC )          AS rn_total_desc -- 案件分案降序排序
                       ,row_number()OVER (PARTITION BY ref_id ORDER BY create_time )               AS rn_total_asc -- 案件分案升序排序
                       ,row_number()OVER (PARTITION BY ref_id ,trigger ORDER BY create_time DESC ) AS rn_ingroup_desc -- 案件组内分案降序排序
                       ,row_number()OVER (PARTITION BY ref_id ,trigger ORDER BY create_time )      AS rn_ingroup_asc -- 案件组内分案升序排序
                FROM `banda-etl-s3`.t_case_distribution_log new
                WHERE topic='AVERAGE_ASSIGN_CASE_TO_REVIEWER' 
            )assign
            LEFT JOIN `banda-etl-s3`.t_admin admin
            ON admin.id=assign.collector_id
            LEFT JOIN
            (select*from banda_rpt_mid.enum_mapping_load_table
                WHERE id='1001'
            ) map_load
            ON assign.trigger=map_load.enum
            LEFT JOIN
            (select*from (
                SELECT  trigger
                       ,description
                       ,row_number()over(partition by trigger ORDER BY update_time desc)rn
                FROM `banda-etl-s3`.t_trigger_group )
                WHERE rn=1 
            ) trigger_group
            ON assign.trigger=trigger_group.trigger
            LEFT JOIN `banda-etl-s3`.t_loan_app app
            ON assign.loan_app_id=app.id
            LEFT JOIN `banda-etl-s3`.t_collection_blacklist blacklist
            ON blacklist.customer_id=app.customer_id AND blacklist.status='ACTIVE'
            LEFT JOIN
            (
                SELECT  rel.loan_id
                FROM `banda-etl-s3`.t_loan_tag_rel rel
                WHERE rel.tag_id IN ( SELECT id FROM `banda-etl-s3`.t_tag WHERE type='SELLING_CASE' )
                AND rel.status='ACTIVE' 
            ) mba
            ON mba.loan_id=assign.loan_app_id
            WHERE mba.loan_id IS NULL -- 剔除出售资产
            and blacklist.customer_id IS NULL-- 剔除黑名单""",
        "coll_30p_paidoff_detail_daily":"""
        
                -- 创建全量刷新表：banda_rpt_mid.coll_30p_paidoff_detail_daily
                -- data_adapundi

                SELECT  date(t3.create_time+interval'7'hour)AS paid_off_time
                       ,date(t3.due_date+interval'7'hour) AS due_date
                       ,t3.reviewer_id                    AS collector_id
                       ,t3.full_name_x                    AS collector_name
                       ,month_d1_dpd_paidoff
                       ,assign_stage
                       ,COUNT(t3.count)                   AS paid_off_count
                       ,SUM(t3.principal_paid)            AS paid_off_principal
                       ,now()                             AS insert_time
                FROM
                (
                    SELECT  t2.id
                           ,t2.reviewer_id
                           ,t2.full_name_x
                           ,t2.due_date
                           ,t2.create_time
                           ,t2.assigntime
                           ,t2.count
                           ,t2.principal_paid
                           ,assign_stage
                           ,CASE WHEN datediff(date_trunc('month',create_time+interval '7' hour),date(due_date+interval'7'hour))>=180 THEN 180
                              WHEN datediff(date_trunc('month',create_time+interval '7' hour),date(due_date+interval'7'hour))>=90 THEN 90
                             WHEN datediff(date_trunc('month',create_time+interval '7' hour),date(due_date+interval'7'hour))>=61 THEN 61
                             WHEN datediff(date_trunc('month',create_time+interval '7' hour),date(due_date+interval'7'hour))>=32 THEN 32 
                             ELSE 31 END AS month_d1_dpd_paidoff -- 结清当月1号逾期天数
                    FROM
                    (
                        SELECT  t1.id
                               ,t1.reviewer_id
                               ,t1.full_name_x
                               ,t1.create_time
                               ,t1.assigntime
                               ,lpay.due_date
                               ,assign_stage
                               ,COUNT(CASE WHEN deposit.create_time>=t1.assigntime AND deposit.create_time<=t1.create_time THEN t1.id ELSE NULL END)AS count
                               ,SUM(CASE WHEN deposit.create_time>=t1.assigntime AND deposit.create_time<=t1.create_time THEN cg.principal ELSE 0 END) AS principal_paid
                        FROM
                        (
                            SELECT  APP.id
                                   ,assign1.collector_id AS reviewer_id
                                   ,assign1.full_name_x
                                   ,app.paid_off_mode
                                   ,lg.create_time
                                   ,assign2.create_time  AS assigntime
                                   ,assign1.assign_stage
                            FROM `banda-etl-s3`.t_loan_app app
                            JOIN
                            (SELECT*FROM banda_rpt_mid.t_assign_detail_stage_daily
                                WHERE rn_total_desc=1 
                            ) assign1
                            ON assign1.loan_app_id=app.id -- 业绩计算给最后分案的人
                            LEFT JOIN
                            (SELECT*FROM banda_rpt_mid.t_assign_detail_stage_daily
                                WHERE rn_ingroup_asc=1 
                            ) assign2
                            ON assign1.loan_app_id=assign2.loan_app_id AND assign1.assign_stage=assign2.assign_stage
                            LEFT JOIN `banda-etl-s3`.t_loan_app_status_log lg
                            ON lg.loan_app_id=app.id
                            WHERE lg.new_status='PAID_OFF'
                            AND app.status IN ('CURRENT', 'GRACE_PERIOD', 'OVERDUE', 'PAID_OFF')
                            AND date(lg.create_time+interval'7'hour)>=date'2021-08-01'
                            AND assign2.create_time<lg.create_time --结清时间大于首次进入队列时间
                            AND datediff(date(now()+interval'7'hour), date(lg.create_time+interval'7'hour))>0 -- 今日之前还款 
                        )t1
                        LEFT JOIN `banda-etl-s3`.t_lpay lpay
                        ON lpay.loan_app_id=t1.id
                        LEFT JOIN `banda-etl-s3`.t_lpay_deposit deposit
                        ON deposit.lpay_id=lpay.id AND deposit.status='CLEARED'
                        LEFT JOIN `banda-etl-s3`.t_clear_detail_log cg
                        ON cg.deposit_id=deposit.id
                        WHERE datediff(DATE(t1.create_time+interval'7'hour), DATE(lpay.due_date+interval'7'hour))>30
                        GROUP BY  t1.id
                                 ,t1.reviewer_id
                                 ,t1.full_name_x
                                 ,t1.create_time
                                 ,t1.assigntime
                                 ,lpay.due_date
                                 ,assign_stage
                    )t2
                )t3
                GROUP BY  date(t3.create_time+interval'7'hour)
                         ,date(t3.due_date+interval'7'hour)
                         ,t3.reviewer_id
                         ,t3.full_name_x
                         ,month_d1_dpd_paidoff
                         ,assign_stage  """
    }
   
}
def make_col_temptable(tableType,tableNm):
    sql=tablemap[tableType][tableNm]
    tablepath="s3://rupiahplus-data-warehouse/etl/"+tableType+"/report_temp_table/"+tableNm
    spark.sql(sql).write.mode("overwrite").orc(tablepath)
#     builtins
# 
if __name__ == "builtins":
    spark = SparkSession\
        .builder\
        .appName("Python Demo")\
        .config("hive.metastore.client.factory.class", "com.amazonaws.glue.catalog.metastore.AWSGlueDataCatalogHiveClientFactory") \
        .enableHiveSupport()\
        .getOrCreate()
#     spark.conf.set("spark.scheduler.mode","FAIR")
    spark.conf.set("spark.sql.shuffle.partitions",20)
    for tableType in tablemap:
#         databasesql="show tables in "+databaseName
#         tables=spark.sql(databasesql)
#         tablelist=tables.collect();
        executor=None
        with futures.ThreadPoolExecutor(max_workers=1) as executor:
            futures_result=futures.wait([executor.submit(make_col_temptable,tableType, tableNm) for tableNm in tablemap[tableType]])
            for  future in futures_result[0]:
                print(future.result())
           


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

None

In [3]:
df=spark.sql("""
  -- 创建全量刷新表：banda_rpt_mid.coll_30p_paidoff_detail_daily
        -- data_adapundi

        SELECT  date(t3.create_time+interval'7'hour)AS paid_off_time
               ,date(t3.due_date+interval'7'hour) AS due_date
               ,t3.reviewer_id                    AS collector_id
               ,t3.full_name_x                    AS collector_name
               ,month_d1_dpd_paidoff
               ,assign_stage
               ,COUNT(t3.count)                   AS paid_off_count
               ,SUM(t3.principal_paid)            AS paid_off_principal
               ,now()                             AS insert_time
        FROM
        (
            SELECT  t2.id
                   ,t2.reviewer_id
                   ,t2.full_name_x
                   ,t2.due_date
                   ,t2.create_time
                   ,t2.assigntime
                   ,t2.count
                   ,t2.principal_paid
                   ,assign_stage
                   ,CASE WHEN datediff(date_trunc('month',create_time+interval '7' hour),date(due_date+interval'7'hour))>=180 THEN 180
                      WHEN datediff(date_trunc('month',create_time+interval '7' hour),date(due_date+interval'7'hour))>=90 THEN 90
                     WHEN datediff(date_trunc('month',create_time+interval '7' hour),date(due_date+interval'7'hour))>=61 THEN 61
                     WHEN datediff(date_trunc('month',create_time+interval '7' hour),date(due_date+interval'7'hour))>=32 THEN 32 
                     ELSE 31 END AS month_d1_dpd_paidoff -- 结清当月1号逾期天数
            FROM
            (
                SELECT  t1.id
                       ,t1.reviewer_id
                       ,t1.full_name_x
                       ,t1.create_time
                       ,t1.assigntime
                       ,lpay.due_date
                       ,assign_stage
                       ,COUNT(CASE WHEN deposit.create_time>=t1.assigntime AND deposit.create_time<=t1.create_time THEN t1.id ELSE NULL END)AS count
                       ,SUM(CASE WHEN deposit.create_time>=t1.assigntime AND deposit.create_time<=t1.create_time THEN cg.principal ELSE 0 END) AS principal_paid
                FROM
                (
                    SELECT  APP.id
                           ,assign1.collector_id AS reviewer_id
                           ,assign1.full_name_x
                           ,app.paid_off_mode
                           ,lg.create_time
                           ,assign2.create_time  AS assigntime
                           ,assign1.assign_stage
                    FROM `banda-etl-s3`.t_loan_app app
                    JOIN
                    (SELECT*FROM banda_rpt_mid.t_assign_detail_stage_daily
                        WHERE rn_total_desc=1 
                    ) assign1
                    ON assign1.loan_app_id=app.id -- 业绩计算给最后分案的人
                    LEFT JOIN
                    (SELECT*FROM banda_rpt_mid.t_assign_detail_stage_daily
                        WHERE rn_ingroup_asc=1 
                    ) assign2
                    ON assign1.loan_app_id=assign2.loan_app_id AND assign1.assign_stage=assign2.assign_stage
                    LEFT JOIN `banda-etl-s3`.t_loan_app_status_log lg
                    ON lg.loan_app_id=app.id
                    WHERE lg.new_status='PAID_OFF'
                    AND app.status IN ('CURRENT', 'GRACE_PERIOD', 'OVERDUE', 'PAID_OFF')
                    AND date(lg.create_time+interval'7'hour)>=date'2021-08-01'
                    AND assign2.create_time<lg.create_time --结清时间大于首次进入队列时间
                    AND datediff(date(now()+interval'7'hour), date(lg.create_time+interval'7'hour))>0 -- 今日之前还款 
                )t1
                LEFT JOIN `banda-etl-s3`.t_lpay lpay
                ON lpay.loan_app_id=t1.id
                LEFT JOIN `banda-etl-s3`.t_lpay_deposit deposit
                ON deposit.lpay_id=lpay.id AND deposit.status='CLEARED'
                LEFT JOIN `banda-etl-s3`.t_clear_detail_log cg
                ON cg.deposit_id=deposit.id
                WHERE datediff(DATE(t1.create_time+interval'7'hour), DATE(lpay.due_date+interval'7'hour))>30
                GROUP BY  t1.id
                         ,t1.reviewer_id
                         ,t1.full_name_x
                         ,t1.create_time
                         ,t1.assigntime
                         ,lpay.due_date
                         ,assign_stage
            )t2
        )t3
        GROUP BY  date(t3.create_time+interval'7'hour)
                 ,date(t3.due_date+interval'7'hour)
                 ,t3.reviewer_id
                 ,t3.full_name_x
                 ,month_d1_dpd_paidoff
                 ,assign_stage




""")
tableType="banda"
tableNm='coll_30p_paidoff_detail_daily'
tablepath="s3://rupiahplus-data-warehouse/etl/"+tableType+"/report_temp_table/"+tableNm
# df.show()
df.repartition(1).write.mode("overwrite").orc(tablepath)
# 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
spark.sql("""

      
""").show(10,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+----------+------------+------------------------------+--------------------+------------+--------------+------------------+-----------------------+
|paid_off_time|due_date  |collector_id|collector_name                |month_d1_dpd_paidoff|assign_stage|paid_off_count|paid_off_principal|insert_time            |
+-------------+----------+------------+------------------------------+--------------------+------------+--------------+------------------+-----------------------+
|2021-08-14   |2021-05-05|102297      |SSS-Q5-004                    |61                  |Q5          |1             |1400000.00000000  |2021-08-26 03:38:41.454|
|2021-08-05   |2021-06-29|102432      |Hendra Ramadhona Ramses-Q4    |32                  |Q4          |1             |720000.00000000   |2021-08-26 03:38:41.454|
|2021-08-07   |2021-06-13|102284      |SSS-Q4-003                    |32                  |Q4          |1             |1510000.00000000  |2021-08-26 03:38:41.454|
|2021-08-09   |2021-06

In [22]:
spark.sql("""
   select CASE when datediff(date_trunc('month',create_time+interval 7 hour),date(due_date+interval   7 hour))>=180 THEN 180
              WHEN datediff(date_trunc('month',create_time+interval 7 hour),date(due_date+interval 7 hour))>=90 THEN 90
              WHEN datediff(date_trunc('month',create_time+interval 7 hour),date(due_date+interval 7 hour))>=61 THEN 61
              WHEN datediff(date_trunc('month',create_time+interval 7 hour),date(due_date+interval 7 hour))>=32 THEN 32 
             ELSE 31 END AS month_d1_dpd_paidoff -- 结清当月1号逾期天数
   from   `banda-etl-s3`.t_lpay lpay
"""
         ).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|month_d1_dpd_paidoff|
+--------------------+
|                  31|
|                  31|
|                  31|
|                  31|
|                  31|
|                  31|
|                  31|
|                  31|
|                  31|
|                  31|
|                  31|
|                  31|
|                  31|
|                  31|
|                  31|
|                  31|
|                  31|
|                  31|
|                  31|
|                  31|
+--------------------+
only showing top 20 rows